In [202]:
import numpy as np
import pymorton as pm
import ast
import math

In [10]:
# input files for operation
Rtreefile = input('please input Rtree file:')
Queryfile = input('please input Query file:')

please input Rtree file:Rtree.txt
please input Query file:NNqueries.txt


In [203]:
#read the input txt files
NNqueries = []
with open(Queryfile,'r') as file:
    for line in file:
        new_line = line.split(' ')
        new_line_float = list(map(float, new_line))
        NNqueries.append(new_line_float)


StrRtree = []
with open(Rtreefile,'r') as file:
    for line in file:
        StrRtree.append(line)
    
# convert string of the input Rtree file to list and keep the list hierarchy
def convert_list(lst):
    newlist = []
    for i in lst:
        if isinstance(i, list):
            newlist.append(convert_list(i))
        else:
            newlist.append(i)
    return newlist
    
Rtree = []
for string in StrRtree:
    singlelist = ast.literal_eval(string)
    newlist = convert_list(singlelist)
    Rtree.append(newlist)

In [204]:
# recount the nodes numbers and how many levels for the Rtree
# set the node capacity for the Rtree
nodecapacityM = 20
nodecapacitym = 0.4*nodecapacityM

#calculate the numbers of leaf node in the Rtree
leafnodeNo = sum(1 for nodes in Rtree if nodes[0] == 0)

#calculate the numbers and levels of nonleaf node part of the Rtree
nonleafnodeNo = math.ceil(leafnodeNo/nodecapacityM)
nonleafnode_level = [nonleafnodeNo]
while nonleafnodeNo > 1:
    nonleafnodeNo = math.ceil(nonleafnodeNo/nodecapacityM)
    nonleafnode_level.append(nonleafnodeNo)

Rtreelevel = len(nonleafnode_level)+1

#print(nonleafnode_level)
NLN_level = nonleafnode_level[::-1]
#print(NLN_level)

In [205]:
#rebuild Rtree
NewRtree = []

for i in NLN_level:
    nonleafnode_list = Rtree[-i:]
    del Rtree[-i:]
    NewRtree.append(nonleafnode_list)
    
NewRtree.append(Rtree)

In [206]:
# calculate euclidean distance of the MBR to the query point
def check_distance(Dq,MBR):
    Xd = Dq[0]
    Yd = Dq[1]
    
    X1 = MBR[1][0]
    Y1 = MBR[1][2]
    X2 = MBR[1][1]
    Y2 = MBR[1][3]
    
    Xn = max(X1, min(Xd, X2))
    Yn = max(Y1, min(Yd, Y2))
    
    Distx = Xn - Xd
    Disty = Yn - Yd
    return np.sqrt(Distx**2 + Disty**2)

In [207]:
# define the k Nearest Neighbors query by reach the closest node MBRs in each level
def kNN_rtree(lst,Dq,k):
    global index
    
    if len(lst) != 0:
        newheap = []
        for i in index:
            for j in lst[0][i][2]:
                distance = check_distance(Dq, j)
                newheap.append((distance,j[0]))
                newheap = sorted(newheap, key=lambda x:x[0])
                if len(newheap)>k:
                    del newheap[-1]
        heapindex = []
        for i in newheap:
            heapindex.append(i[1])
        
    del lst[0]
    return heapindex

In [208]:
DQid = 0
for Dq in NNqueries:
    kNNRtree = NewRtree[:]
    
    index = [0]
    k = 10
    
    for i in range(len(NewRtree)):
        heap_index = kNN_rtree(kNNRtree, Dq, k)
        index = heap_index
        
    print(str(DQid)+'('+str(len(index))+'): '+','.join(str(x) for x in index))
    DQid += 1

0(10): 9311,7001,803,5361,6764,3905,1642,3260,4669,5762
1(10): 6764,1642,9311,7001,4888,3260,3905,4669,803,5361
2(10): 9311,7001,803,5361,6764,3905,1642,3260,5762,4669
3(10): 6764,1642,9311,7001,4888,3260,3905,4669,803,5887
4(10): 803,9311,5361,7001,6764,4669,1642,6595,9874,3905
5(10): 9311,6764,7001,1642,803,4669,3905,3260,4888,5361
6(10): 6764,1642,9311,7001,4669,4888,3260,3905,803,5361
7(10): 6764,4669,1642,9311,7001,4888,6595,803,9874,5361
8(10): 9311,803,7001,5361,6764,1642,3905,4669,3260,5762
9(10): 6764,1642,9311,7001,4669,4888,3260,3905,803,5361
10(10): 9311,803,6764,7001,5361,4669,1642,6595,3905,9874
11(10): 803,6764,4669,9311,5361,7001,6595,1642,9874,7588
12(10): 9311,803,6764,5361,7001,4669,1642,6595,9874,3905
13(10): 6764,9311,7001,1642,4669,803,4888,3905,3260,5361
14(10): 6764,9311,1642,7001,4669,4888,803,3260,3905,5361
15(10): 9311,803,6764,7001,5361,4669,1642,6595,9874,3905
16(10): 803,5361,9311,7001,6764,4669,1642,6595,9874,5762
17(10): 9311,7001,6764,1642,803,3905,3260

In [185]:
Dq = NNqueries[0]
ScanRtree = NewRtree[:]
index = [0]
k = 10


In [189]:
b,c = scan_rtree(ScanRtree, Dq, k)
index = b
print(b)
print(c)
print(len(ScanRtree))

[9311, 7001, 803, 5361, 6764, 3905, 1642, 3260, 4669, 5762]
[(0.15, 9311), (0.19, 7001), (0.24, 803), (0.29, 5361), (0.35, 6764), (0.36, 3905), (0.37, 1642), (0.38, 3260), (0.46, 4669), (0.48, 5762)]
0
